K-neighbors es lo recomendado para medir similaridad en datasets grandes por su eficiencia. Además, si se quiere extender la funcionalidad hacia user-item recommendation, es ideal también. En este modelo se puede definir distintas medidas de similaridad como distancia euclidiana o cosine similarity. 
Lo más importante es escoger correctamente los features a ingresar en la matriz, deben ser relevantes. -- Falta revisar si el método tfidf tiene un tope de palabras y si se puede asignar stopwords en español. 


-- Columnas a usar: genres, developer, tags, specs

In [2]:
import pandas as pd

In [3]:
games = pd.read_csv('datasets/steam_games.csv')

In [4]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.2+ MB


In [19]:
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')
games['release_year'] = games['release_date'].dt.year

In [21]:
games['release_year'] = games['release_year'].fillna(0).astype(int)
games['release_year']

0        2018
1        2018
2        2017
3        2017
4           0
         ... 
32130    2018
32131    2018
32132    2018
32133    2017
32134       0
Name: release_year, Length: 32135, dtype: int64

In [22]:
games['release_year'] = games['release_year'].astype('str').replace('0', '')
games['release_year']

0        2018
1        2018
2        2017
3        2017
4            
         ... 
32130    2018
32131    2018
32132    2018
32133    2017
32134        
Name: release_year, Length: 32135, dtype: object

In [24]:
games.isnull().sum()

publisher       8071
genres          3283
app_name           2
title           2050
url                0
release_date    2352
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
release_year       0
dtype: int64

In [27]:
games.fillna('', inplace=True)

In [29]:
games.isnull().sum()

publisher          0
genres             0
app_name           0
title              0
url                0
release_date    2352
tags               0
reviews_url        0
specs              0
price              0
early_access       0
id                 0
developer          0
release_year       0
dtype: int64

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Combine relevant columns into a single 'profile'
games['profile'] = games['genres'] + " " + games['tags'] + " " + games['specs'] + " " + games['developer'] + " " + games['release_year']
games['profile'] = games['profile'].fillna('')

# Create a TF-IDF Vectorizer to transform the 'profile' into numerical data
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(games['profile'])

# Use k-NN to find similar games
knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  # Using 6 neighbors to include the game itself
knn.fit(tfidf_matrix)

# Create a function to get recommendations
def recommend(game_title):
    idx = games.index[games['title'] == game_title].tolist()[0]
    distances, indices = knn.kneighbors(tfidf_matrix[idx])
    
    # Exclude the first (index 0) since it'll be the game itself
    game_indices = indices[0][1:]
    return games['title'].iloc[game_indices]

# Test
print(recommend("Counter-Strike: Global Offensive"))


1868                                            Insurgency
32012                               Counter-Strike: Source
32117                       Counter-Strike: Condition Zero
12867    ELEAGUE 2018 Boston CS:GO Major Championship M...
29344                                Call of Duty®: Ghosts
Name: title, dtype: object


In [ ]:
# Convertir todas las listas en string para asegurarnos que esté tomando todas
# Pasar todo a .lower()
# Usar el id del videojuego en vez del titulo
# Probar con diferente combinación de columnas 

In [37]:
games['profile']

0        ['Action', 'Casual', 'Indie', 'Simulation', 'S...
1        ['Free to Play', 'Indie', 'RPG', 'Strategy'] [...
2        ['Casual', 'Free to Play', 'Indie', 'Simulatio...
3        ['Action', 'Adventure', 'Casual'] ['Action', '...
4         ['Action', 'Indie', 'Casual', 'Sports'] ['Sin...
                               ...                        
32130    ['Casual', 'Indie', 'Simulation', 'Strategy'] ...
32131    ['Casual', 'Indie', 'Strategy'] ['Strategy', '...
32132    ['Indie', 'Racing', 'Simulation'] ['Indie', 'S...
32133    ['Casual', 'Indie'] ['Indie', 'Casual', 'Puzzl...
32134     ['Early Access', 'Adventure', 'Indie', 'Actio...
Name: profile, Length: 32135, dtype: object